<a href="https://colab.research.google.com/github/Nishidh25/Covid-19-Detection-from-Lung-X-rays/blob/master/Covid_19_Detection_from_Lung_X_rays_Web_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir /content/static
!mkdir /content/templates

!wget https://raw.githubusercontent.com/Nishidh25/Covid-19-Detection-from-Lung-X-rays/master/templates/image_predicted.html
!cp image_predicted.html /content/templates
!wget https://raw.githubusercontent.com/Nishidh25/Covid-19-Detection-from-Lung-X-rays/master/templates/main.html
!cp main.html /content/templates

!wget https://github.com/Nishidh25/Covid-19-Detection-from-Lung-X-rays/raw/master/model.h5

--2020-06-10 16:07:30--  https://raw.githubusercontent.com/Nishidh25/Covid-19-Detection-from-Lung-X-rays/master/templates/image_predicted.html
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 486 [text/plain]
Saving to: ‘image_predicted.html’

image_predicted.htm 100%[===================>]     486  --.-KB/s    in 0s      

2020-06-10 16:07:30 (22.8 MB/s) - ‘image_predicted.html’ saved [486/486]

--2020-06-10 16:07:34--  https://raw.githubusercontent.com/Nishidh25/Covid-19-Detection-from-Lung-X-rays/master/templates/main.html
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awai

In [2]:
!pip install flask_ngrok
from flask_ngrok import run_with_ngrok
from flask import Flask, request, render_template, flash, redirect, url_for
from werkzeug.utils import secure_filename
import tensorflow as tf
import numpy as np
import os

In [0]:
def predict(file):  
  img = tf.io.read_file(file)
  img = tf.image.decode_jpeg(img, channels=3)
  img.set_shape([None, None, 3])
  img = tf.image.resize(img, (224, 224))
  img = np.array(img) # convert to numpy array
  img = np.expand_dims(img, 0) # make 'batch' of 1
  img = img /255. # Rescaling
  model =  tf.keras.models.load_model("model.h5")
  pred = model.predict(img,verbose=1)
  print(pred)
  if pred > 0.5 :
    return "Positive"
  else : return "Negative"

In [4]:
UPLOAD_FOLDER = '/content/static'
ALLOWED_EXTENSIONS = {'jpg', 'jpeg'}
app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
run_with_ngrok(app)

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS


@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        # check if the post request has the file part
        if 'file' not in request.files:
            return render_template('main.html')
        file = request.files['file']
        # if user does not select file, browser also
        # submit an empty part without filename
        if file.filename == '':
            return render_template('main.html')
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            image_dir = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            pred = predict(image_dir)
            print(pred)
            return render_template("image_predicted.html", user_image = file.filename, prediction = pred )
    return render_template('main.html')
if __name__ == '__main__':
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://e7ba47c80eba.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [10/Jun/2020 16:08:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2020 16:08:07] "GET /favicon.ico HTTP/1.1" 404 -


1/1 [==============================] - 0s 2ms/step


...

127.0.0.1 - - [10/Jun/2020 16:08:16] "GET /static/345db0222228e0b5f49d346b38aa90_jumbo.jpeg HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2020 16:08:18] "GET / HTTP/1.1" 200 -


1/1 [==============================] - 0s 2ms/step


127.0.0.1 - - [10/Jun/2020 16:08:24] "POST / HTTP/1.1" 200 -


[[3.0035064e-05]]
Negative


127.0.0.1 - - [10/Jun/2020 16:08:25] "GET /static/IM-0001-0001.jpeg HTTP/1.1" 200 -
